In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

In [2]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened
DEBUG:Cloud:Deleting 760.lock (7.00012148329 days old)
DEBUG:Cloud:Deleting 760.log (7.00012148329 days old)


In [3]:
A = mg.load_dataset('table_A')
B = mg.load_dataset('table_B')

In [4]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'zipcode', 'zipcode', ['name'], ['name'])

In [25]:
f = mg.get_features_for_matching(A, B)

In [27]:
mg.init_jvm()

True

In [63]:
D = mg.extract_feat_vecs(C, feature_table=f)

In [30]:
import pandas as pd

In [64]:
D.iat[0, 5] = pd.np.NaN

In [59]:
D

,_id,ltable.ID,rtable.ID,ID_ID_lev,ID_ID_jar,ID_ID_jwn,ID_ID_sdx,ID_ID_exm,ID_ID_jac_qgm_3_qgm_3,name_name_jac_qgm_3_qgm_3,...,name_name_swg,birth_year_birth_year_exm,birth_year_birth_year_anm,birth_year_birth_year_lev,hourly_wage_hourly_wage_exm,hourly_wage_hourly_wage_anm,hourly_wage_hourly_wage_lev,zipcode_zipcode_exm,zipcode_zipcode_anm,zipcode_zipcode_lev
0,0,a1,b1,0.5,0.666667,NaN,0.888889,0,0,0.000000,...,0.327273,0,0.998994,0.75,0,0.983333,0.25,1,1,1
1,1,a1,b2,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.145455,0,0.998492,0.75,0,0.937500,0.75,1,1,1
2,2,a1,b6,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.181818,0,0.998994,0.75,0,0.923077,0.50,1,1,1
3,3,a3,b1,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.127273,0,0.999497,0.75,0,0.921875,0.25,1,1,1
4,4,a3,b2,0.0,0.000000,0.000000,0.888889,0,0,0.400000,...,0.781818,1,1.000000,1.00,1,1.000000,1.00,1,1,1
5,5,a3,b6,0.0,0.000000,0.000000,0.888889,0,0,0.043478,...,0.400000,0,0.999497,0.75,0,0.984615,0.75,1,1,1
6,6,a2,b3,0.0,0.000000,0.000000,0.888889,0,0,0.388889,...,0.707692,1,1.000000,1.00,1,1.000000,1.00,1,1,1
7,7,a2,b4,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.181818,0,0.996982,0.75,0,0.945455,0.50,1,1,1
8,8,a2,b5,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.123077,0,0.997988,0.75,0,0.785714,0.25,1,1,1
9,9,a4,b3,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.166667,0,0.999497,0.75,0,0.846154,0.50,1,1,1


In [33]:
E = D[f['feature_name']]

In [35]:
V = E.values

In [36]:
from sklearn.preprocessing import Imputer

In [65]:
def impute(table, f, missing_val='NaN',
           strategy='mean', axis=0, val_all_nans=0):
    table = table.copy()
    tbl = table[f]
    
    t = tbl.values
    print t[0]
    imp = Imputer(missing_values=missing_val, strategy=strategy, axis=axis)
    imp.fit(t)
    imp.statistics_[pd.np.isnan(imp.statistics_)] = val_all_nans
    t = imp.transform(t)
    table[f] = t
    return table

In [66]:
impute(D, f['feature_name'])

[ 0.5         0.66666669         nan  0.88888884  0.          0.          0.
  0.          0.          0.32727273  0.          0.5         0.18181819
  0.32727271  0.          0.99899447  0.75        0.          0.98333333
  0.25        1.          1.          1.        ]


,_id,ltable.ID,rtable.ID,ID_ID_lev,ID_ID_jar,ID_ID_jwn,ID_ID_sdx,ID_ID_exm,ID_ID_jac_qgm_3_qgm_3,name_name_jac_qgm_3_qgm_3,...,name_name_swg,birth_year_birth_year_exm,birth_year_birth_year_anm,birth_year_birth_year_lev,hourly_wage_hourly_wage_exm,hourly_wage_hourly_wage_anm,hourly_wage_hourly_wage_lev,zipcode_zipcode_exm,zipcode_zipcode_anm,zipcode_zipcode_lev
0,0,a1,b1,0.5,0.666667,0.095238,0.888889,0,0,0.000000,...,0.327273,0,0.998994,0.75,0,0.983333,0.25,1,1,1
1,1,a1,b2,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.145455,0,0.998492,0.75,0,0.937500,0.75,1,1,1
2,2,a1,b6,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.181818,0,0.998994,0.75,0,0.923077,0.50,1,1,1
3,3,a3,b1,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.127273,0,0.999497,0.75,0,0.921875,0.25,1,1,1
4,4,a3,b2,0.0,0.000000,0.000000,0.888889,0,0,0.400000,...,0.781818,1,1.000000,1.00,1,1.000000,1.00,1,1,1
5,5,a3,b6,0.0,0.000000,0.000000,0.888889,0,0,0.043478,...,0.400000,0,0.999497,0.75,0,0.984615,0.75,1,1,1
6,6,a2,b3,0.0,0.000000,0.000000,0.888889,0,0,0.388889,...,0.707692,1,1.000000,1.00,1,1.000000,1.00,1,1,1
7,7,a2,b4,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.181818,0,0.996982,0.75,0,0.945455,0.50,1,1,1
8,8,a2,b5,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.123077,0,0.997988,0.75,0,0.785714,0.25,1,1,1
9,9,a4,b3,0.0,0.000000,0.000000,0.888889,0,0,0.000000,...,0.166667,0,0.999497,0.75,0,0.846154,0.50,1,1,1


In [62]:
pd.np.mean(D['ID_ID_jwn'])

0.095238098076411662